In [ ]:
# Config and imports

import pypsa
import logging
import atlite
import time
import warnings
import geopandas as gpd
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
from plotly import express as px
from plotly import graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.gridspec import GridSpec
import ipywidgets as widgets
from atlite.gis import shape_availability, ExclusionContainer

from library.weather import generate_cutout
from library.geography import availability_matrix, capacity_factor

logging.basicConfig(level=logging.INFO)

In [ ]:
LAN_CODE = "14"
KOM_CODE = "1480"
WEATHER_START = "2023-01"
WEATHER_END = "2023-12"

[CUTOUT, SELECTION, eez, index] = generate_cutout(LAN_CODE, KOM_CODE, WEATHER_START, WEATHER_END, "./data", "SA")

In [ ]:
# Exclude land use per solar/wind

# Source: https://www.uts.edu.au/oecm/renewable-resource-mapping
# (Classification mapping: https://collections.sentinel-hub.com/corine-land-cover/readme.html)
# With the exception of "41; Water bodies"

# Land-cover classes NOT included for solar areas:
EXCLUDED_SOLAR = [1,4,5,6,7,9,10,11,23,24,25,27,30,33,34,35,36,37,38,39,40,41,42,43,44] # 1.1.1,1.2.2,1.2.3,1.2.4,1.3.1,1.3.3,1.4.1,1.4.2,3.1.1,3.1.2,3.1.3,3.2.2,3.3.1,3.3.4,3.3.5,4.1.1,4.1.2,4.2.1,4.2.2,4.2.3,5.1.1,5.1.2,5.2.1,5.2.2,5.2.3
# Land-cover classes NOT included for wind energy areas:
EXCLUDED_WIND_NON_OCEAN = [1,2,3,4,5,6,7,9,10,11,15,16,17,23,24,25,27,30,31,33,34,35,36,37,38,39,40,41,42,43,44] # 1.1.1,1.1.2,1.2.1,1.2.2,1.2.3,1.2.4,1.3.1,1.3.3,1.4.1,1.4.2,2.2.1,2.2.2,2.2.3,3.1.1,3.1.2,3.1.3,3.2.2,3.3.1,3.3.2,3.3.4,3.3.5,4.1.1,4.1.2,4.2.1,4.2.2,4.2.3,5.1.1,5.1.2,5.2.1,5.2.2,5.2.3
INCLUDED_WIND_OCEAN = [44, 41] # 5.2.3, 5.1.2


CORINE = "./data/geo/corine.tif"
solar_excluder = ExclusionContainer()
onwind_excluder = ExclusionContainer()
offwind_excluder = ExclusionContainer()

solar_excluder.add_raster(CORINE, codes=EXCLUDED_SOLAR)
onwind_excluder.add_raster(CORINE, codes=EXCLUDED_WIND_NON_OCEAN)
offwind_excluder.add_raster(CORINE, codes=INCLUDED_WIND_OCEAN, invert=True)

solar_avail = CUTOUT.availabilitymatrix(SELECTION, solar_excluder)
onwind_avail = CUTOUT.availabilitymatrix(SELECTION, onwind_excluder)
offwind_avail = CUTOUT.availabilitymatrix(SELECTION, offwind_excluder)

AVAIL_CAPACITY_SOLAR = capacity_factor(CUTOUT, SELECTION, "solar", "", "./data")
AVAIL_CAPACITY_ONWIND = capacity_factor(CUTOUT, SELECTION, "onwind", "2030_5MW_onshore.yaml", "./data")
AVAIL_CAPACITY_OFFWIND = capacity_factor(CUTOUT, SELECTION, "offwind", "2030_20MW_offshore.yaml", "./data")


In [ ]:
## Parameters
window_size=56 # Rolling average window size for charts


## Create solar availability map widget
solar_map = widgets.Output()
with solar_map:
    solar_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Reds" })
    plt.show()

## Create onwind availability map widget
onwind_map = widgets.Output()
with onwind_map:
    onwind_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Greens" })
    plt.show()

## Create offwind availability map widget
offwind_map = widgets.Output()
with offwind_map:
    offwind_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Blues" })
    plt.show()

## Create solar capacity factor line chart widget
solar_capacity_line_chart = widgets.Output()
with solar_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_SOLAR.plot(ax=ax, color="Red", alpha=0.2)
    AVAIL_CAPACITY_SOLAR.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Red")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create onwind capacity factor line chart widget
onwind_capacity_line_chart = widgets.Output()
with onwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_ONWIND.plot(ax=ax, color="Green", alpha=0.2)
    AVAIL_CAPACITY_ONWIND.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Green")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create offwind capacity factor line chart widget
offwind_capacity_line_chart = widgets.Output()
with offwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_OFFWIND.plot(ax=ax, color="Blue", alpha=0.2)
    AVAIL_CAPACITY_OFFWIND.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Blue")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Build the dashboard and show

sun_wind_map = widgets.HBox([solar_map, onwind_map, offwind_map])

row_1 = widgets.HBox([sun_wind_map])
row_2 = widgets.HBox([solar_capacity_line_chart])
row_3 = widgets.HBox([onwind_capacity_line_chart])
row_4 = widgets.HBox([offwind_capacity_line_chart])

dashboard = widgets.VBox([row_1, row_2, row_3, row_4])

dashboard
